# Small Local Experimentation with Detectron2 

This notebook details how to train Detectron2 object detection locally -- with the purpose of running small experiments on a local GPU before kicking off a larger training job. 

In [ ]:
%%bash 

# Install the dependencies locally to run Detectron2 on a Linux laptop 

pip install torchvision torch --upgrade 
pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI' 'git+https://github.com/facebookresearch/fvcore' 'git+https://github.com/facebookresearch/detectron2.git' google-colab scikit-image sagemaker-inference

In [ ]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data.catalog import MetadataCatalog, DatasetCatalog
import os

train_dataset_name = "train"
train_dataset_location = "../data/"
train_annotation_file = "train.json"
train_image_dir = "training_set"

register_coco_instances(train_dataset_name, {}, os.path.join(train_dataset_location, train_annotation_file), 
                        os.path.join(train_dataset_location, train_image_dir))

cb_train_meta = MetadataCatalog.get(train_dataset_name)

val_dataset_name = "val"
val_dataset_location = "../data/"
val_annotation_file = "val.json"
val_image_dir = "validation_set"

register_coco_instances(val_dataset_name, {}, os.path.join(val_dataset_location, val_annotation_file), 
                        os.path.join(val_dataset_location, val_image_dir))

cb_val_meta = MetadataCatalog.get(val_dataset_name)

In [ ]:
# Print meta info about our dataset
cb_train_meta

If you are interested to see the input of our model, feel free to execute cell below. Here, dataset_dicts provides a mapping between images and groundtruth values (bounding boxes and segmentation masks)

In [ ]:
train_dataset_dicts = DatasetCatalog.get(train_dataset_name)
print(dataset_dicts)

Now, let's visualize our input images and groundtruth bounding boxes and segementation masks.

In [ ]:
import random
import cv2
from detectron2.utils.visualizer import Visualizer
from google.colab.patches import cv2_imshow
 
for d in random.sample(train_dataset_dicts, 3):
    print(d["file_name"])
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=cb_train_meta, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

## Transfer learn a small Detectron2 model locally

Let's first check if we can fine tune model locally. Training your model locally may be a good choice in case of small models and small datasets. In our cases, let's test that model can perform finetuning on custom dataset for limited number of iterations. 

If this is successfull, we'll then go ahead and train model for larger number of iterations across multiple nodes.

In [ ]:
# Do initial configuration
from detectron2.config import get_cfg
from detectron2 import model_zoo

config_file = "COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"
# config_file = "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"

# COCO Pretrained weight URLs 
# 101: https://dl.fbaipublicfiles.com/detectron2/COCO-Detection/faster_rcnn_R_101_FPN_3x/137851257/model_final_f6e8b1.pkl
# 50: https://dl.fbaipublicfiles.com/detectron2/COCO-Detection/faster_rcnn_R_50_FPN_1x/137257794/model_final_b275ba.pkl

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(config_file))
cfg.DATASETS.TRAIN = (train_dataset_name,)
cfg.DATASETS.TEST = (val_dataset_name,)  # no metrics implemented for this dataset
cfg.MODEL.WEIGHTS = "https://dl.fbaipublicfiles.com/detectron2/COCO-Detection/faster_rcnn_R_101_FPN_3x/137851257/model_final_f6e8b1.pkl"
cfg.DATALOADER.NUM_WORKERS = 4
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = (2500)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (
    32
)  # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5
cfg.MODEL.BACKBONE.FREEZE_AT = 5

cfg.OUTPUT_DIR = "local_model_output"

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [ ]:
from detectron2.engine import DefaultTrainer

trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

## Run inference with the locally trained model! 

In [ ]:
from detectron2.engine import DefaultPredictor

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
cfg.DATASETS.TEST = (val_dataset_name, )
predictor = DefaultPredictor(cfg)

In [ ]:
cb_train_meta

In [ ]:
from detectron2.utils.visualizer import ColorMode

test_pics_dir = '<PATH_TO_SOME_TEST_IMGS>'

for img_ in os.listdir(test_pics_dir): 
    img_name = test_pics_dir + img_ 

    im = cv2.imread(img_name)
    outputs = predictor(im)

    v = Visualizer(im[:, :, ::-1],
                   metadata=cb_train_meta, 
                   scale=0.8)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])